In [1]:
seed = 0

In [2]:
targets = ['gender', 'profession','ideology_binary', 'ideology_multiclass']

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sentence_transformers import SentenceTransformer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score

def f1s(preds, dtrain):
    f1 = f1_score(dtrain.get_label(), np.round(preds), average='macro')
    return 'f1', f1

def get_train_test(data_train, reduce=False):
    users_train, users_test = train_test_split(data_train.index.unique(), test_size=.2, random_state=19970808)
    
    if reduce:
        data_train['idd'] = np.array(range(data_train.shape[0])) % reduce
        df_tweets = data_train.reset_index().groupby(by=['label', 'idd']).tweet.apply(lambda x: ' '.join(x))
        df = data_train.reset_index().groupby(by=['label', 'idd']).first()
        
    else:
        df = data_train
        df_tweets = data_train.tweet
    
    df_targets = df[targets].apply(LabelEncoder().fit_transform)
    
    X_train = df_tweets.loc[users_train]
    Y_train = df_targets.loc[users_train]
    Y_train = Y_train.droplevel(1)

    X_test = df_tweets.loc[users_test]
    Y_test = df_targets.loc[users_test]
    Y_test = Y_test.droplevel(1)
    
    return X_train, X_test, Y_train, Y_test

/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
data_pre = pd.read_csv("../data/raw/tweets_complete.csv")
data_pre.set_index('label', inplace=True)

data_train = pd.read_csv("../data/raw/training.csv")
data_dev = pd.read_csv('../data/raw/development.csv')
data_dev_test = pd.read_csv('../data/raw/development_test.csv')
data_test = pd.read_csv("../data/raw/test_without_labels.csv")
data_train.set_index('label', inplace=True)

data_train['tweet'] = data_pre['tweet_emoji_signos_pre']

X_train, X_test, Y_train, Y_test = get_train_test(data_train, reduce=4)

In [ ]:
#bert_model = SentenceTransformer('finiteautomata/beto-sentiment-analysis')
#bert_model = SentenceTransformer('dccuchile/bert-base-spanish-wwm-uncased')

X_train_emb = bert_model.encode(X_train.tolist(), device='cuda', show_progress_bar=True)
X_test_emb = bert_model.encode(X_test.tolist(), device='cuda', show_progress_bar=True)

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
def test_model(model, X_train, X_test):
    print(model)
    for target in targets:
        y_train = Y_train[target]
        y_test = Y_test[target]


        model.fit(X_train, y_train)

        y_pred = model.predict_proba(X_test)[:,1]
        y_pred = pd.Series(y_pred, Y_test.index)
        result = f1_score(y_test.groupby(level=0).mean(), y_pred.groupby(level=0).mean().round(), average='macro')

        print('f1 {}: {}'.format(target, result))

In [11]:
model = LogisticRegression(max_iter=2000)
test_model(model, X_train_emb, X_test_emb)

NameError: name 'X_train_emb' is not defined

In [453]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA
parameters = [{'pca__n_components':[10,30,40,50,80,100,200]}]
model =  Pipeline([('pca', PCA()), ('qda', QuadraticDiscriminantAnalysis())])
model = GridSearchCV(model, cv=4, verbose=False, n_jobs=-1, param_grid=parameters)

test_model(model, X_train_emb, X_test_emb)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('qda',
                                        QuadraticDiscriminantAnalysis())]),
             n_jobs=-1,
             param_grid=[{'pca__n_components': [10, 30, 40, 50, 80, 100, 200]}],
             verbose=False)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers

/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


f1 profession: 0.8898601398601398
f1 ideology_binary: 0.7731481481481481


/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are colline

f1 ideology_multiclass: 0.2686688311688311


In [456]:
parameters = [{'n_estimators':[100,200,300,400,500], 'max_depth':[4,8]}]
model = XGBClassifier(use_label_encoder=False)
model = GridSearchCV(model, cv=4, verbose=False, n_jobs=-1, param_grid=parameters)

test_model(model, X_train_emb, X_test_emb)

GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
    

KeyboardInterrupt: 

## TFIDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [6]:
from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

In [12]:
tf = TfidfVectorizer(analyzer = 'word',
                     min_df = 1,
                     max_features = 5000,
                     lowercase=True,
                     stop_words=stopwords)
X_train_tf = tf.fit_transform(X_train).toarray()
X_test_tf = tf.transform(X_test).toarray()

In [ ]:
np.linspace()

In [15]:
np.logspace(-3, 3, 10)

array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])

In [33]:
from scipy.stats import loguniform
a = 

1.352179960158715

In [34]:
param_grid_reg = [
    {"C": [.0001, .001, .001, 0.1, 1, 2, 5, 10],
     "penalty": ["l1", "l2"]
     }
    ]

model = LogisticRegression(max_iter=2000)
model = GridSearchCV(model, param_grid=param_grid_reg, cv=5, verbose=False, n_jobs=-1)

test_model(model, X_train_tf, X_test_tf)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=2000), n_jobs=-1,
             param_grid=[{'C': [0.0001, 0.001, 0.001, 0.1, 1, 2, 5, 10],
                          'penalty': ['l1', 'l2']}],
             verbose=False)


/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/azaelc

f1 gender: 0.6203730272596844


/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/azaelc

f1 profession: 0.8138455044322499


/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/azaelc

f1 ideology_binary: 0.9027777777777779


/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/azaelc

f1 ideology_multiclass: 0.29583333333333334


In [442]:
sd = StandardScaler()

In [443]:
model =  Pipeline([('pca', PCA()), ('qda', QuadraticDiscriminantAnalysis())])
model = GridSearchCV(model, cv=4, verbose=False, n_jobs=-1, param_grid=parameters)

X_train_tf_sd = sd.fit_transform(X_train_tf)
X_test_tf_sd = sd.transform(X_test_tf)
test_model(model, X_train_tf_sd, X_test_tf_sd)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('qda',
                                        QuadraticDiscriminantAnalysis())]),
             n_jobs=-1,
             param_grid=[{'pca__n_components': [10, 30, 40, 50, 80, 100, 200]}],
             verbose=False)
f1 gender: 0.4490740740740741
f1 profession: 0.8622448979591837
f1 ideology_binary: 0.9176470588235295


/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


f1 ideology_multiclass: 0.30293757649938796


In [446]:
model = RandomForestClassifier(n_estimators=300)
test_model(model, X_train_tf_sd, X_test_tf_sd)

RandomForestClassifier(n_estimators=300)
f1 gender: 0.6159554730983303
f1 profession: 0.5178571428571428
f1 ideology_binary: 0.8673684210526316
f1 ideology_multiclass: 0.19285714285714284


In [444]:
from sklearn.ensemble import RandomForestClassifier

In [356]:
X_train_full = np.concatenate((X_train_emb, X_train_tf.toarray()), axis=1)
X_test_full = np.concatenate((X_test_emb, X_test_tf.toarray()), axis=1)
model = XGBClassifier(use_label_encoder=False, 
                      learning_rate=.01,
                      max_depth=3,
                      gamma=1,
                      colsample_bytree=.8,
                      n_estimators=200)

eval_set = [(X_train_full, gender_train), (X_test_full, gender_test)]

model.fit(X_train_full, gender_train, eval_set=eval_set, eval_metric=f1s)

gender_pred = model.predict_proba(X_test_full)
gender_pred = gender_pred[:,1]
gender_pred = pd.Series(gender_pred, gender_test.index)

f1_score(gender_test.groupby(level=0).mean(), gender_pred.groupby(level=0).mean().round(), average='macro')

/home/azaelcarrillo/miniconda3/envs/rest-mex-reco-gpu/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.69161	validation_0-f1:0.65524	validation_1-logloss:0.69183	validation_1-f1:0.55544
[1]	validation_0-logloss:0.69004	validation_0-f1:0.70769	validation_1-logloss:0.69073	validation_1-f1:0.62083
[2]	validation_0-logloss:0.68854	validation_0-f1:0.67603	validation_1-logloss:0.68946	validation_1-f1:0.59501
[3]	validation_0-logloss:0.68705	validation_0-f1:0.68212	validation_1-logloss:0.68836	validation_1-f1:0.57915
[4]	validation_0-logloss:0.68557	validation_0-f1:0.69229	validation_1-logloss:0.68710	validation_1-f1:0.59966
[5]	validation_0-logloss:0.68412	validation_0-f1:0.69563	validation_1-logloss:0.68602	validation_1-f1:0.59937
[6]	validation_0-logloss:0.68281	validation_0-f1:0.69088	validation_1-logloss:0.68516	validation_1-f1:0.60421
[7]	validation_0-logloss:0.68141	validation_0-f1:0.69280	validation_1-logloss:0.68415	validation_1-f1:0.60727
[8]	validation_0-logloss:0.68004	validation_0-f1:0.68746	validation_1-logloss:0.68316	validation_1-f1:0.59966
[9]	valida

0.7214854111405835

In [324]:
X_train_tf.shape

(2000, 10000)

In [357]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tf, gender_train)
gender_pred1 = model.predict_proba(X_test_tf)

gender_pred1 = gender_pred1[:,1]
gender_pred1 = pd.Series(gender_pred1, gender_test.index)

f1_score(gender_test.groupby(level=0).mean(), gender_pred1.groupby(level=0).mean().round(), average='macro')

0.6293079293919865

In [358]:
train_pred1 = model.predict_proba(X_train_tf)
train_pred1 = train_pred1[:,1]
train_pred1 = pd.Series(train_pred, gender_train.index)

/tmp/ipykernel_563395/1528668345.py:3: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  train_pred1 = pd.Series(train_pred, gender_train.index)


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
gender_pred2 = pd.concat([gender_pred, gender_pred1])
tmp = gender_pred2.groupby(level=0).apply(list)
X_test_sup = pd.DataFrame(tmp.to_list(), index=tmp.index)

gender_train2 = pd.concat([train_pred, train_pred1])
tmp = gender_train2.groupby(level=0).apply(list)
X_train_sup = pd.DataFrame(tmp.to_list(), index=tmp.index)

In [359]:
model = LogisticRegression()
model.fit(X_train_sup, gender_train.groupby(level=0).mean())
gender_pred_final = model.predict(X_test_sup)

In [322]:
f1_score(gender_test.groupby(level=0).mean(), gender_pred_final, average='macro')

0.6634615384615385